<a href="https://colab.research.google.com/github/enesdemirag/gate-detection/blob/colab/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
# Clone git repository
!git clone https://github.com/enesdemirag/gate-detection.git

Cloning into 'gate-detection'...
remote: Enumerating objects: 858, done.
remote: Total 858 (delta 0), reused 0 (delta 0), pack-reused 858
Receiving objects: 100% (858/858), 262.06 MiB | 7.51 MiB/s, done.
Resolving deltas: 100% (328/328), done.


In [40]:
# Install requirements
!pip3 install Cython
!pip3 install contextlib2
!pip3 install pillow
!pip3 install lxml
!pip3 install jupyter
!pip3 install matplotlib

In [41]:
# Checkout simple-training branch
import os
os.chdir('gate-detection/')
!git checkout model

Branch 'model' set up to track remote branch 'model' from 'origin'.
Switched to a new branch 'model'


In [42]:
# Clone object detection API
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 33257 (delta 0), reused 8 (delta 0), pack-reused 33249
Receiving objects: 100% (33257/33257), 511.78 MiB | 12.43 MiB/s, done.
Resolving deltas: 100% (21241/21241), done.
Checking out files: 100% (3190/3190), done.


In [43]:
# Install COCO API
!git clone https://github.com/cocodataset/cocoapi.git
os.chdir('cocoapi/PythonAPI/')
!make
!cp -r pycocotools ../../models/research/

Cloning into 'cocoapi'...
remote: Enumerating objects: 959, done.
remote: Total 959 (delta 0), reused 0 (delta 0), pack-reused 959
Receiving objects: 100% (959/959), 11.69 MiB | 5.33 MiB/s, done.
Resolving deltas: 100% (570/570), done.
python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/gate-detection/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/pycocotools
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6

In [0]:
# Compile Protobuf
os.chdir('../../models/research/')
!protoc object_detection/protos/*.proto --python_out=.

# Add libraries to Python Path
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
# !set PYTHONPATH=/content/gate-detection/models/research : /content/gate-detection/models/research/slim

In [57]:
# Install API
!sudo python3 setup.py install

running install
running bdist_egg
running egg_info
writing object_detection.egg-info/PKG-INFO
writing dependency_links to object_detection.egg-info/dependency_links.txt
writing requirements to object_detection.egg-info/requires.txt
writing top-level names to object_detection.egg-info/top_level.txt
writing manifest file 'object_detection.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/object_detection
copying build/lib/object_detection/model_lib.py -> build/bdist.linux-x86_64/egg/object_detection
copying build/lib/object_detection/model_lib_test.py -> build/bdist.linux-x86_64/egg/object_detection
creating build/bdist.linux-x86_64/egg/object_detection/data_decoders
copying build/lib/object_detection/data_decoders/__init__.py -> build/bdist.linux-x86_64/egg/object_detection/data_decoders
copying build/lib/object_detection/data_decoders/tf_example_d

In [0]:
# Copy training files into API folder
os.chdir('../../')
!cp -r data/ models/research/object_detection/
!cp -r images/ models/research/object_detection/
!cp -r model/ models/research/object_detection/
!cp -r training/ models/research/object_detection/

In [0]:
# Get inside object_detection folder
os.chdir('models/research/object_detection/')

In [60]:
# Run training script
!python3 legacy/train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/ssd_mobilenet.config

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Traceback (most recent call last):
  File "legacy/train.py", line 52, in <module>
    from object_detection.builders import model_builder
  File "/usr/local/lib/python3.6/dist-packages/object_detection-0.1-py3.6.egg/object_detection/builders/model_builder.py", line 35, in <module>
    from object_detection.models import faster_rcnn_inception_resnet_v2_feature_extractor as frcnn_inc_res
  File "/usr/local/lib/python3.6/dist-packages/object_detection-0.1-py3.6.egg/object_detection/models/faster_rcnn_inception_resnet_v2_feature_extractor.py", line 30, in <module>
    from nets import inception_resnet_v2
ModuleNotFoundError: No modu